# BUILD MFCC

In [1]:
import numpy as np
import librosa
import pandas as pd
from sklearn import preprocessing
from tqdm.notebook import trange, tqdm

### GLOBAL VARIABLES

In [2]:
INPUT_PATH = 'data/processed/X_slice.npy'
OUTPUT_PATH = 'data/features/X_mfcc_002.npy'
SR = 22050

### FUNCTIONS

In [3]:
def wav2mfcc(wave, sr, n_fft, hop_length, n_mfcc, max_pad_len, normalize, padding_mode, **kwargs):

    #wave, sr = librosa.load(wav_file, mono=True)
    
    #if 0 < len(wave): # workaround: 0 length causes error
    #    wave, _ = librosa.effects.trim(wave) # trim, top_db=default(60)

    if normalize:
        wave = librosa.util.normalize(wave) # normalizing data before mfcc

    # making mfcc from signal
    S = librosa.feature.mfcc(y=wave, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc, **kwargs)

    if max_pad_len:
        if S.shape[1] > max_pad_len:
            S = S[:,:max_pad_len]
        else:
            pad_width = max_pad_len - S.shape[1]
            S = np.pad(S, pad_width=((0, 0), (0, pad_width)), mode=padding_mode)
   
    return S

In [4]:
def gen_mfcc(input_path, output_path, sr, n_fft=2048, hop_length=512, n_mfcc=128, max_pad_len=None, normalize=True, padding_mode='constant', **kwargs):
    
    # Load wave files
    wavfiles = np.load(input_path)
    
    # Init Mel vectors
    X_mfcc = []
    
    for idx in trange(wavfiles.shape[0]):
        X_mfcc.append(wav2mfcc(wavfiles[idx, :], sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc, max_pad_len=max_pad_len, 
                             normalize=normalize, padding_mode=padding_mode, **kwargs))
        pass
    
    X_mfcc = np.array(X_mfcc)
    
    # Saving Mels 
    np.save(output_path, X_mfcc)
    
    pass

### RUNING

In [5]:
gen_mfcc(INPUT_PATH, OUTPUT_PATH, SR, n_fft=2048, hop_length=512, n_mfcc=60, normalize=True)